In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from pandas_datareader import data as pdr
import yfinance as yf
yf.pdr_override()

In [4]:
ele_gas = ('9501.T', '9502.T', '9503.T', '9504.T', '9505.T', '9506.T', '9507.T',
            '9508.T', '9509.T', '9511.T', '9513.T', '9517.T', '9519.T',
              '9531.T', '9532.T', '9533.T', '9534.T', '9535.T', '9536.T',
              '9543.T', '9551.T')

In [5]:
def calculate_returns(prices):
    return (prices / prices.shift(1) - 1) * 100

In [9]:
def get_data(tickers):
    df = pd.DataFrame()
    for ticker in tickers:
        print(ticker)
        data = pdr.get_data_yahoo(ticker,start = '2006-01-01',end='2024-09-09')['Adj Close']
        data.name = ticker
        df = pd.concat([df,data],axis = 1)
        
    for i in np.arange(len(df.index)):
        df.index.values[i] = str(df.index[i].date())

    df.index = df.index.rename('Date')
    
    return df

In [19]:
elec_gas_return = get_data(ele_gas)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

9501.T
9502.T
9503.T



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

9504.T
9505.T
9506.T



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


...


[*********************100%%**********************]  1 of 1 completed

9536.T



[*********************100%%**********************]  1 of 1 completed


9543.T
9551.T


[*********************100%%**********************]  1 of 1 completed


In [21]:
elec_gas_return.dropna(axis=1, how="any", inplace=True)
elec_gas_return.head()

,9501.T,9502.T,9503.T,9504.T,9505.T,9506.T,9507.T,9508.T,9509.T,9511.T,9513.T,9531.T,9532.T,9533.T,9534.T,9535.T,9536.T,9543.T
Date,,,,,,,,,,,,,,,,,,
2006-01-02,2560.274658,1742.818604,1701.020386,1301.841431,1592.149170,1634.630493,1694.237549,1777.323242,1746.473633,1167.949219,1683.131348,1779.984863,1368.234863,1786.663452,1220.289917,381.605957,1693.903076,605.227722
2006-01-03,2560.274658,1742.818604,1701.020386,1301.841431,1592.149170,1603.981567,1694.237549,1777.323242,1746.473633,1167.949219,1683.131348,1779.984863,1368.234863,1786.663452,1220.289917,381.605957,1693.903076,605.227722
2006-01-04,2546.869873,1733.515625,1711.085449,1307.538330,1595.566040,1648.252563,1708.326904,1784.265747,1753.750488,1308.425903,2014.770264,1800.366699,1378.320435,1813.104126,1226.815674,367.446289,1725.155762,605.970459
2006-01-05,2578.147217,1745.919556,1717.795410,1316.084229,1605.815674,1655.063721,1718.893921,1808.565186,1761.027466,1353.816528,2099.550537,1847.924194,1391.767944,1820.659058,1223.552979,366.738281,1712.654419,605.227722
2006-01-06,2582.615234,1755.222412,1707.730225,1333.176636,1622.899780,1658.468994,1729.460815,1812.036377,1786.496826,1340.002075,2134.459961,1810.557373,1408.575684,1862.209717,1226.815674,366.738281,1731.406372,604.485291


In [25]:
import itertools
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.vector_ar.vecm import coint_johansen
import statsmodels.api as sm


In [24]:
pairs = []
i = 1
for comb in itertools.combinations(elec_gas_return.columns,2):
    prices = elec_gas_return.loc[:,comb]
    #print(df_train.loc[:,comb])
    prices = prices.dropna()
    try:
        # JohansenTestResult
        jrt = coint_johansen(prices,k_ar_diff=0,det_order=-1)
        if jrt.lr1[0] > jrt.cvt[0,2] and jrt.lr1[1] < jrt.cvt[1,0]\
        and jrt.lr2[0] > jrt.cvm[0,2] and jrt.lr2[1] < jrt.cvm[1,0]:
            pairs.append(comb)
    except ValueError as error:
        print(i,comb)
    i += 1
        
print(pairs)

[('9502.T', '9503.T'), ('9502.T', '9534.T'), ('9504.T', '9534.T'), ('9505.T', '9534.T'), ('9506.T', '9534.T'), ('9511.T', '9532.T'), ('9511.T', '9534.T'), ('9513.T', '9534.T'), ('9531.T', '9535.T'), ('9532.T', '9535.T'), ('9532.T', '9536.T'), ('9533.T', '9534.T'), ('9534.T', '9535.T'), ('9534.T', '9536.T'), ('9535.T', '9543.T')]


In [26]:
def plot_price_and_residual(df_train, pair):
    #get the names of the stocks
    stock1 = pair[0]
    stock2 = pair[1]
    
    X = df_train[stock1]
    Y = df_train[stock2]
    #X = sm.add_constant(X)
    
    #linear regression with X and Y.
    model = sm.OLS(Y, X).fit()
    residuals = model.resid
    mean_residuals = residuals.mean()
    std_residuals = residuals.std()
    fig, axs = plt.subplots(2, 1, figsize=(12, 10), sharex=True)

    #plot prices
    axs[0].plot(X.index, X, label=stock1, color='blue')
    axs[0].plot(Y.index, Y, label=stock2, color='orange')
    axs[0].set_title('Stock Prices')
    axs[0].set_ylabel('Price')
    axs[0].legend()

    #plot residual, its mean, and mean +_ 2 * std
    axs[1].plot(residuals, label='Residuals')
    axs[1].axhline(mean_residuals, color='red', linestyle='-', linewidth=1, label='Mean')
    axs[1].axhline(mean_residuals + 2 * std_residuals, color='green', linestyle='--', linewidth=1, label='Mean + 2 Std')
    axs[1].axhline(mean_residuals - 2 * std_residuals, color='orange', linestyle='--', linewidth=1, label='Mean - 2 Std')
    axs[1].set_title('Residuals of Linear Regression')
    axs[1].set_xlabel('Date')
    axs[1].set_ylabel('Residuals')
    axs[1].legend()

    plt.tight_layout()
    plt.show()

In [ ]:
plot_price_and_residual(elec_gas_return, pairs[1])